In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from rdkit import Chem
from rdkit.Chem import inchi
from tqdm import tqdm
from time import sleep
from tqdm.notebook import tqdm

from chembl_structure_pipeline import standardizer as ChEMBL_standardizer
from papyrus_structure_pipeline import standardize

In [ ]:
kpuu_raw =  pd.read_excel('Kpuu_raw.xlsx')

In [ ]:
kpuu_raw.head()

In [ ]:
def compare_values(row):
    if row['Kp,uu,brain'] > 0.33:
        return 'active'
    elif row['Kp,uu,brain'] <= 0.1:
        return 'inactive'
    else:
        return '?'

kpuu_raw['status_activity'] = kpuu_raw.apply(compare_values, axis=1)

kpuu_raw.head(10)

In [ ]:
kpuu_raw = kpuu_raw.rename(columns={'SMILES': 'SMILES_raw'})

columns_to_keep = ['SMILES_raw', 'status_activity', 'species']
kpuu = kpuu_raw[columns_to_keep]


In [ ]:
#Remove rows without SMILES

def remove_nan_smiles(df):

    df = df[~(df['SMILES_raw'].isna())]
    df = df.reset_index(drop=True)

    return df

In [ ]:
#Papyrus Standardization

def create_sd_smiles(sd_mol):
    try:
        standardized_smiles =  Chem.MolToSmiles(sd_mol)
        return standardized_smiles
    except Exception as e:
        print(f"An sd_smiles error occurred: {str(e)}")
        return None
    
#Create InChI keys from standardized molecules
def mol_to_inchi_key(sd_mol):
    if sd_mol is not None:

        inchi_str = inchi.MolToInchi(sd_mol)
        inchi_key = inchi.InchiToInchiKey(inchi_str)
    else:
        inchi_key = None   
    return inchi_key

def standardize_molecule(mol):
    standardized_mol =  standardize(mol,raise_error=False )
    return standardized_mol

#Standardize 

def standardize_workflow(df_raw):
    for i in range(0,len(df_raw)):
        smiles =df_raw.at[i,'SMILES_raw']
        mol = Chem.MolFromSmiles(smiles)
        sd_mol =  standardize_molecule(mol)
        sd_smiles = create_sd_smiles(sd_mol)
        sd_inchi_key = mol_to_inchi_key(sd_mol)
        df_raw.at[i,'papyrus_SMILES'] = sd_smiles
        df_raw.at[i,'papyrus_inchi_key'] = sd_inchi_key

    print(f'df length after standardization: {len(df_raw)}')

    return df_raw

In [ ]:
#Check for missing inchi key

def missing_inchi(df_raw):
    smiles_nan = df_raw['papyrus_SMILES'].isna().sum()
    inchikey_nan =df_raw['papyrus_inchi_key'].isna().sum()
    print(f'DB length: {len(df_raw)},        SMILES nan: {smiles_nan},        inchi key nan: {inchikey_nan}')

    #Remove rows with missing inchikey
    df_valid_inchi= df_raw[((df_raw['papyrus_inchi_key'].notna()))]
    print('-----remove missing inchikey----')
    print(f'updated length: {len(df_valid_inchi)}')

    return df_valid_inchi

In [ ]:
def inchi_first_part(inchi):
    return inchi.split('-')[0]

def create_connectivity_inchi(df):
    df['inchi_connectivity'] = df['papyrus_inchi_key'].apply(inchi_first_part)
    
    return df

In [ ]:
#Check for duplicates

def remove_duplicates(df):
    print(f'length: {len(df)}')
    inchi_un = df['inchi_connectivity'].nunique()
    print(f'unique_inchi: {inchi_un}')


    unique_counts = df.groupby('inchi_connectivity')['status_activity'].nunique()
    contradicting_duplicates = unique_counts[unique_counts > 1].index

    print(f'Contradicting duplicates: {len(contradicting_duplicates)}')

    # Separate contradicting duplicates into a new dataframe
    duplicates_df = df[df['inchi_connectivity'].isin(contradicting_duplicates)].copy()

    # Remove contradicting duplicates from the original dataframe
    df = df[~df['inchi_connectivity'].isin(contradicting_duplicates)].copy()

    print(f'Original dataframe after contradicting duplicates removed: {len(df)}')
    print(f'Original dataframe unique_inchi after removal: {df["inchi_connectivity"].nunique()}')

    df=df.drop_duplicates(subset=['inchi_connectivity'], keep="first").reset_index(drop=True)
    
    print(f'Original dataframe after non-contradicting duplicates removed: {len(df)}')

    return df, duplicates_df

In [ ]:
df = kpuu

df = remove_nan_smiles(df)

df_sd = standardize_workflow(df)
df_sd.to_csv('kpuu_standardized_val.csv')

df_valid = missing_inchi(df_sd)
df_valid.to_csv('kpuu_have_inchi_key_val.csv')

df_connectivity_inchi = create_connectivity_inchi(df_valid)
df_connectivity_inchi.to_csv('kpuu_connectivity_inchi_val.csv')



In [ ]:
df_no_duplicates, df_contra_duplicates = remove_duplicates(df_connectivity_inchi)
df_no_duplicates.to_csv('kpuu_no_duplicates_val.csv')
df_contra_duplicates.to_csv('kpuu_contradicting_duplicates.csv')

Further curate contradicting


In [ ]:
#Further curate contradicting


print(f'length: {len(df)}')

unique_counts = df_contra_duplicates.groupby('inchi_connectivity')['species'].nunique()
same_species = unique_counts[unique_counts >  1].index

print(f'Dupicates with same species: {len(same_species)}')


# Create a mask to identify the first entries of duplicates with the same species
df_unique = df_contra_duplicates.drop_duplicates(subset=['inchi_connectivity', 'species'], keep='first').reset_index(drop=True)

print(f'Length after inchi-key AND species duplicates are removed: {len(df_unique)}')



In [ ]:
df_unique.to_csv('kpuu_contradicting_duplicates_once.csv')

Remove training molecules

In [ ]:
influx = pd.read_csv('../../../3_Combined/31_Combined_datasets/1_data_curation/influx/combined_influx_train_raw.csv')
efflux = pd.read_csv('../../../3_Combined/31_Combined_datasets/1_data_curation/efflux/combined_efflux_train_raw.csv')
pampa = pd.read_csv('../../../3_Combined/31_Combined_datasets/1_data_curation/pampa/combined_pampa_train_raw.csv')
bbb = pd.read_csv('../../../3_Combined/31_Combined_datasets/1_data_curation/bbb/combined_bbb_train_raw.csv')

In [ ]:
mask = df_no_duplicates['inchi_connectivity'].isin(influx['inchi_connectivity'])
common_values_list = df_no_duplicates[mask]['inchi_connectivity'].tolist()
print(f'Common molecules {len(common_values_list)}')
kpuu_val = df_no_duplicates[~mask]
kpuu_val = kpuu_val.reset_index(drop=True)
print(f'After removed: {len(kpuu_val)}')

In [ ]:
kpuu_val.head()

In [ ]:
mask = kpuu_val['inchi_connectivity'].isin(efflux['inchi_connectivity'])
common_values_list = kpuu_val[mask]['inchi_connectivity'].tolist()
print(f'Common molecules {len(common_values_list)}')
kpuu_val = kpuu_val[~mask]
kpuu_val = kpuu_val.reset_index(drop=True)
print(f'After removed: {len(kpuu_val)}')

In [ ]:
mask = kpuu_val['inchi_connectivity'].isin(pampa['inchi_connectivity'])
common_values_list = kpuu_val[mask]['inchi_connectivity'].tolist()
print(f'Common molecules {len(common_values_list)}')
kpuu_val = kpuu_val[~mask]
kpuu_val = kpuu_val.reset_index(drop=True)
print(f'After removed: {len(kpuu_val)}')

In [ ]:
mask = kpuu_val['inchi_connectivity'].isin(bbb['inchi_connectivity'])
common_values_list = kpuu_val[mask]['inchi_connectivity'].tolist()
print(f'Common molecules {len(common_values_list)}')
kpuu_val = kpuu_val[~mask]
kpuuval = kpuu_val.reset_index(drop=True)
print(f'After removed: {len(kpuu_val)}')

In [ ]:
kpuu_val.to_csv('kpuu_validation_not_everything_classified.csv')

Keep only classified

In [ ]:
kpuu_val_classified = kpuu_val[kpuu_val['status_activity'].isin(['active', 'inactive'])]
len(kpuu_val_classified)

In [ ]:
kpuu_val_classified.to_csv('kpuu_validation.csv', index=True)